  ## Linear Regression - Categorical Predictor Inference


  ## Description of the Data
  These data contain information on mother's and baby's health for 1,174 pregnant women.

In [ ]:
library(tidyverse)
library(ggformula)
library(mosaic)

theme_set(theme_bw(base_size = 18))

baby <- read_csv("https://raw.githubusercontent.com/lebebr01/statthink/master/data-raw/baby.csv")
baby <- baby %>%
  mutate(smoker = ifelse(maternal_smoker, 1, 0))
head(baby)

  ## Linear Regression - Categorical Predictor Refresher
  Now it is time to fit a model to the data here to explore if there indeed is a difference in the population. We know descriptively there is a difference in the two group means and medians, but is this difference large enough to be practical? The model is fitted similar to before with the `lm()` function and a similar formula as before. The outcome (birth weight) is to the left of the `~` and the predictor (maternal smoking status) is to the right.

In [ ]:
smoker_reg <- lm(birth_weight ~ maternal_smoker, data = baby)
coef(smoker_reg)

 ## Inference
  Similar to the continuous predictor, resampling/bootstrap takes a similar method in the case with a single categorical predictor.

  In order to get some sense of the amount of error in the estimate of the linear slope here, a bootstrap can be done to provide some evidence of the likely range of slope values. The bootstrap will take the following general steps:
  1. Resample the observed data available, with replacement
  2. Fit the same linear regression model as above.
  3. Save the slope coefficient representing the relationship between birth weight and gestational days
  4. Repeat steps 1 - 3 many times
  5. Explore the distribution of slope estimates from the many resampled data sets.

In [ ]:
resample_baby <- function(...) {
  baby_resample <- baby %>%
    sample_n(nrow(baby), replace = TRUE)

  baby_resample %>%
    lm(birth_weight ~ maternal_smoker, data = .) %>%
    coef(.) %>%
    .[2] %>%
    data.frame()
}

resample_baby()

  Now that there is a function that does steps 1 - 3, these processes can now be repeated many times.

In [ ]:
baby_coef <- map(1:10000, resample_baby) %>%
  bind_rows()
names(baby_coef) <- 'slope'

gf_density(~ slope, data = baby_coef)

In [ ]:
baby_coef %>%
  df_stats(~ slope, quantile(c(0.05, 0.5, 0.95)))

 ## More than 2 categorical groups
 Before the model contained one attribute that represented two groups. What happens when there are more than two groups for an attribute? To explore this, the college scorecard data will be used again.

In [ ]:
college_score <- read_csv("https://raw.githubusercontent.com/lebebr01/statthink/master/data-raw/College-scorecard-4143.csv", guess_max = 10000)
head(college_score)

 ### Explore distribution 3 groups
 Early in the course, the distribution of admission rates by the primary degree that the institution grants was explored. Below is a violin plot that shows these three distributions.

In [ ]:
gf_violin(adm_rate ~ preddeg, data = college_score, fill = 'gray85', 
          size = 1, draw_quantiles = c(0.1, 0.5, 0.9))

 There may be some small differences between these groups, but more formally we can test this to understand the amount of uncertainty in the average of the distributions. This again will make use of the `lm()` function in R and the formula is very similar to what was done before and mimics the formula from the violin plot above.

In [ ]:
adm_model <- lm(adm_rate ~ preddeg, data = college_score)
coef(adm_model)

 Guesses as to what these coefficients represent? How were the categorical groups turned into the different elements in the model?

 ## Overall model fit
 There is a measure of overall model fit that is commonly used in the research literature for linear regression models, called R-squared. R-squared represents the proportion of variation in the outcome that is explained by the attributes in the model. The statistic ranges from 0 to 1 where values closer to 1 indicate larger percentages of variation explained. This can be extracted from the model directly.

In [ ]:
summary(adm_model)$r.squared

 Another one can be computed from the baby data where the birth weight was the outcome and gestational days was the primary attribute used as a predictor.

In [ ]:
baby_reg <- lm(birth_weight ~ gestational_days, data = baby)
summary(baby_reg)$r.squared

 For models with a single predictor variable, R-squared is the correlation coefficient squared. For example:

In [ ]:
cor(birth_weight ~ gestational_days, data = baby) ^ 2

### Residual Standard Error

Another metric to evaluate model fit is a statistic called the residual standard error. This statistic is interpreted just like the standard deviation, but instead of being the average distance data are from the mean, the residual standard error is the average distance data points are from the regression line. 

Similar to the standard deviation, in general, smaller values of the residual standard error are better. However, the scale of this statistic is dependent on the scale of the outcome. 

In [ ]:
summary(baby_reg)$sigma

In [ ]:
gf_point(birth_weight ~ gestational_days, data = baby) %>%
   gf_smooth(method = 'lm', size = 1) %>%
   gf_labs(y = "Birth Weight", 
                  x = "Gestational Days")

In [ ]:
summary(adm_model)$sigma